In [ ]:
# print out the coverage rate for all 5-node & 6-node patterns for all possible hyper-tree decomposition

import os
import networkx as nx
from networkx.algorithms.approximation import treewidth
from networkx.algorithms.approximation.treewidth import treewidth_min_degree
from itertools import combinations
import matplotlib.pyplot as plt
from pynauty import Graph, autgrp

def nx_to_nauty(G):
    ngraph = Graph(number_of_vertices=len(G.nodes()))
    for u, v in G.edges():
        ngraph.connect_vertex(u, v)
    return ngraph

def is_valid_group(G, group):
    # A group is valid if no two nodes in the group are connected
    for node1, node2 in combinations(group, 2):
        if G.has_edge(node1, node2):
            return False
    return True

def find_all_partitions(G, nodes, current_partition, all_partitions):
    if not nodes:
        all_partitions.append(current_partition)
        return
    
    first_node = nodes[0]
    remaining_nodes = nodes[1:]

    for i in range(len(current_partition)):
        new_group = current_partition[i] | {first_node}
        if is_valid_group(G, new_group):
            find_all_partitions(G, remaining_nodes, current_partition[:i] + [new_group] + current_partition[i+1:], all_partitions)

    # Start a new group with the first node
    find_all_partitions(G, remaining_nodes, current_partition + [{first_node}], all_partitions)

def get_all_partitions(G):
    nodes = list(G.nodes())
    all_partitions = []
    find_all_partitions(G, nodes, [], all_partitions)
    return all_partitions

def is_covered(decomposition, partition):
    for bag in decomposition:
        if len(bag) <= 1:
            continue
        for group in partition:
            if len(group) <= 1:
                continue
            if len(group.intersection(bag)) >= 2:
                return False
    return True

def load_pattern(pattern_dir):
    patterns = {} # {'file_name': pattern}
    for file in os.listdir(pattern_dir):
        path = os.path.join(pattern_dir, file)
        with open(path, "r") as f:
            pattern = nx.Graph()
            num_v, _ = map(int, f.readline().strip().split())
            pattern.add_nodes_from(range(0, num_v))
            for line in f.readlines()[:-1]:
                u, v = line.strip().split()
                pattern.add_edge(int(u), int(v))
        patterns[os.path.splitext(file)[0]] = pattern
    return patterns

def compute_coverage_rate(pattern, verbose=False):
    # print the pattern graph
    if verbose:
        nx.draw(pattern, with_labels=True)
        plt.show()

    # compute tree decomposition with the tree width
    tw, decomposition = treewidth_min_degree(pattern)
    if verbose:
        print("True Treewidth:", tw)
        print("True Tree decomposition:")
        for i, bag in enumerate(decomposition.nodes):
            print(f"Bag {i}: {bag}")
        print()

    # compute possible homorphism sub-pattern of the pattern
    partitions = get_all_partitions(pattern)

    # print all sub-patterns
    for i, partition in enumerate(partitions):
        if verbose:
            print(f"Partition {i+1}: {partition}")

            # draw this patition
            partition_graph = nx.Graph()
            partition_graph.add_nodes_from(range(0, len(partition)))
            for index_1, group_1 in enumerate(partition):
                for index_2, group_2 in enumerate(partition):
                    if group_1 == group_2:
                        continue
                    find_edge = False
                    for node_1 in group_1:
                        for node_2 in group_2:
                            if pattern.has_edge(node_1, node_2):
                                partition_graph.add_edge(index_1, index_2)
                                find_edge = True
                                break
                        if find_edge:
                            break
            nx.draw(partition_graph, with_labels=True)
            plt.show()
    # compute the number of partitions that can be covered
    coverage = 0
    for partition in partitions:
        if is_covered(decomposition, partition):
            coverage += 1
    if verbose:
        print(f"Coverage rate: {coverage}/{len(partitions)}")
    
    if verbose:
        generators, grpsize1, grpsize2, orbits, numorbits = autgrp(nx_to_nauty(pattern))
        print(pattern.nodes())
        for auto in generators:
            print(auto)
        print(grpsize1, grpsize2, orbits, numorbits)

    return coverage, len(partitions), tw

if __name__ == "__main__":

    # create the pattern
    # pattern = nx.Graph()
    # pattern.add_nodes_from([1, 2, 3, 4, 5, 6])
    # pattern.add_edges_from([(1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (3, 4), (4, 5), (2, 6), (3, 6), (4, 6), (5, 6)])

    # load all 5voc, 6voc
    voc5_dir = "exp/pattern_graph/5voc"
    voc6_dir = "exp/pattern_graph/6voc"
    voc5_patterns = load_pattern(voc5_dir)
    voc6_patterns = load_pattern(voc6_dir)

    # compute the coverage rate for all patterns
    info = []
    for name, pattern in voc5_patterns.items():
        coverage, total, tw = compute_coverage_rate(pattern, verbose=True if name =='30' else False)
        info.append((name, coverage, total, tw))

In [ ]:
# find out the file of the pattern in the paper
import networkx as nx

def load_pattern(pattern_dir):
    patterns = {} # {'file_name': pattern}
    for file in os.listdir(pattern_dir):
        path = os.path.join(pattern_dir, file)
        with open(path, "r") as f:
            pattern = nx.Graph()
            num_v, _ = map(int, f.readline().strip().split())
            pattern.add_nodes_from(range(0, num_v))
            for line in f.readlines()[:-1]:
                u, v = line.strip().split()
                pattern.add_edge(int(u), int(v))
        patterns[os.path.splitext(file)[0]] = pattern
    return patterns


if __name__ == "__main__":
    # create the pattern
    # pattern = nx.Graph()
    # pattern.add_nodes_from([1, 2, 3, 4, 5, 6])
    # pattern.add_edges_from([(1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (3, 4), (4, 5), (2, 6), (3, 6), (4, 6), (5, 6)])

    pattern = nx.Graph()
    pattern.add_nodes_from([0, 1, 2, 3, 4])
    pattern.add_edges_from([(0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (3, 4)])

    voc5_patterns = load_pattern(voc5_dir)

    # traverse all patterns and find the iso-match pattern in the paper
    for name, p in voc5_patterns.items():
        if nx.is_isomorphic(pattern, p):
            print(name)
